【4時間でPython AIアプリ開発】Python Django scikit learnでAIアプリを開発  
https://www.youtube.com/watch?v=_87RY9rb79I&list=WL&index=15  

【たった90分でAI開発】Python word2vec + LightGBMで多値分類に挑戦！  
https://www.youtube.com/watch?v=wGCzd2k-0GU

# インポート

In [5]:
import pandas as pd
from collections import Counter
import string
import jctconv
import emoji
import re

# データチェック

In [6]:
!ls

rakuma_NPL_add_w2v_LGBM.ipynb


In [7]:
df = pd.read_csv('../nlp-model/all_rakuma.csv').fillna("")

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122070 entries, 0 to 122069
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   date           122070 non-null  object
 1   item_url       122070 non-null  object
 2   title          122070 non-null  object
 3   price          122070 non-null  int64 
 4   brand          122070 non-null  object
 5   category       122070 non-null  object
 6   good           122070 non-null  int64 
 7   bad            122070 non-null  int64 
 8   normal         122070 non-null  int64 
 9   ship_days      122070 non-null  object
 10  comments       122070 non-null  object
 11  last_log_date  122070 non-null  object
 12  good_counts    122070 non-null  int64 
 13  prefecture     122070 non-null  object
 14  delivery_fee   122070 non-null  object
 15  seller_url     122070 non-null  object
 16  size           122070 non-null  object
 17  picture_url    122070 non-null  object
 18  desc

In [9]:
df.head()

,date,item_url,title,price,brand,category,good,bad,normal,ship_days,comments,last_log_date,good_counts,prefecture,delivery_fee,seller_url,size,picture_url,description,details_dict
0,2019/10/08 23:39:51,https://item.fril.jp/055f4080a2a0ae275db6ba7be...,Epiphone LTD Korina Explorer Bass,40000,Epiphone,楽器 > ベース > エレキベース,109,0,0,1-2日後,,2000-01-01,0,岐阜県,着払い,https://fril.jp/shop/42df208014145c438f96e01f5...,なし,https://img.fril.jp/img/259586636/l/734809353....,コリーナボディ ブラックピックガード ゴールドパーツの豪華なエクスプローラー コリーナ ...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', 'ブランド':..."
1,2019/10/08 23:39:51,https://item.fril.jp/896a1d0197d05cf8431c136d9...,TVB-100 (SUN),38000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,"['TOKYO GUITARS', '本モデルを含め、TOKYO GUITARS工房にて製造...",2019/08/31,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223532100/l/635334018....,Tokyo Guitars モデル名 TVB 100 SUN Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
2,2019/10/08 23:39:51,https://item.fril.jp/6bdb6b4dd0f6c518938b94ec2...,TRB-4000 (BL),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223294381/l/634674588....,Tokyo Guitars モデル名 TRB 4000 BL Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
3,2019/10/08 23:39:51,https://item.fril.jp/b89c82258515163e5aa36aa48...,TRB-4000 (FG),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223294160/l/634673947....,Tokyo Guitars モデル名 TRB 4000 FG Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
4,2019/10/08 23:39:51,https://item.fril.jp/cf904ef67c6cec09a7465e7f4...,TB-100 (BRWN),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/216145047/l/614817207....,Tokyo Guitars モデル名 TB 100 BRWN Tokyo Guit...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."


In [10]:
df.category.nunique()

1012

In [11]:
df.category.value_counts() / len(df) *100

メンズ > 靴/シューズ > スニーカー                 9.174244
エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ    8.975178
エンタメ/ホビー > フィギュア > アニメ/ゲーム           5.418203
エンタメ/ホビー > CD > ポップス/ロック(洋楽)         2.919636
エンタメ/ホビー > フィギュア > 特撮                2.577210
                                       ...   
エンタメ/ホビー > 雑誌 > 絵本/児童書               0.000819
スマホ/家電/カメラ > 冷暖房/空調 > その他            0.000819
メンズ > ジャケット/アウター > ライダースジャケット        0.000819
楽器 > DTM/DAW > DAWソフトウェア             0.000819
エンタメ/ホビー > 漫画 > 女性漫画                 0.000819
Name: category, Length: 1012, dtype: float64

In [12]:
ctg_df = df.category.value_counts().reset_index() #Seriesにresetindexをするとdfになる
ctg_df

,index,category
0,メンズ > 靴/シューズ > スニーカー,11199
1,エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ,10956
2,エンタメ/ホビー > フィギュア > アニメ/ゲーム,6614
3,エンタメ/ホビー > CD > ポップス/ロック(洋楽),3564
4,エンタメ/ホビー > フィギュア > 特撮,3146
...,...,...
1007,エンタメ/ホビー > 雑誌 > 絵本/児童書,1
1008,スマホ/家電/カメラ > 冷暖房/空調 > その他,1
1009,メンズ > ジャケット/アウター > ライダースジャケット,1
1010,楽器 > DTM/DAW > DAWソフトウェア,1


In [13]:
over1000_data = ctg_df[ctg_df.category > 1000]
over1000_data

,index,category
0,メンズ > 靴/シューズ > スニーカー,11199
1,エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ,10956
2,エンタメ/ホビー > フィギュア > アニメ/ゲーム,6614
3,エンタメ/ホビー > CD > ポップス/ロック(洋楽),3564
4,エンタメ/ホビー > フィギュア > 特撮,3146
5,エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー,2780
6,スポーツ/アウトドア > フィッシング > ルアー用品,2562
7,エンタメ/ホビー > トレーディングカード > シングルカード,2521
8,エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー,2162
9,エンタメ/ホビー > CD > アニメ,2020


In [14]:
over1000_data.category.sum()

63006

In [15]:
data = df.merge(over1000_data[['index']], left_on='category', right_on='index')
data

,date,item_url,title,price,brand,category,good,bad,normal,ship_days,...,last_log_date,good_counts,prefecture,delivery_fee,seller_url,size,picture_url,description,details_dict,index
0,2019/10/09 15:12:05,https://item.fril.jp/56e1889edb4373b060739c876...,テレカ 未使用品 仮面ライダー 生誕20周年 東映ビデオ販売 販促用非売品,2500,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,37,0,0,1-2日後,...,2000-01-01,0,神奈川県,送料込,https://fril.jp/shop/d16cd0f0dd452bb98787cc349...,なし,https://img.fril.jp/img/152839505/l/513856974....,テレカ 未使用 50度数 レア 仮面ライダー 誕生20周年 レーザーディスク販売時の販促...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他
1,2019/10/09 15:12:05,https://item.fril.jp/ac2ac2e9968e30e868e1cc87b...,送料無料 新品 DVD Perfume WORLD TOUR 1st 初回,4800,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,60,0,0,1-2日後,...,2000-01-01,0,京都府,送料込,https://fril.jp/shop/a1c14809b40f535a4ec33b213...,なし,https://img.fril.jp/img/154299175/l/439614345....,新品 DVD Perfume WORLD TOUR 1st 初回プレス盤 ご覧頂きあり...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他
2,2019/10/09 15:12:05,https://item.fril.jp/797582a80b28b6c07bfe84c03...,☆ウルトラマン☆レーザーディスク ジャンク？,1980,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,87,0,1,1-2日後,...,2000-01-01,0,岡山県,送料込,https://fril.jp/shop/5d3613f833a4ee3a15e9dd72f...,なし,https://img.fril.jp/img/244479040/l/693281987....,ウルトラマンのレーザーディスクです 再生機器が無いのと 未開封のようですので現状渡しで...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他
3,2019/10/09 15:12:05,https://item.fril.jp/5da5209c6bd09ceef11d1ed2d...,MAISHA （マイシャ）Sadao Watanabe 渡辺貞夫★LD,2600,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,9,0,0,1-2日後,...,2000-01-01,0,東京都,着払い,https://fril.jp/shop/22d352b7a529906589dcdbc60...,なし,https://img.fril.jp/img/124947216/l/353856650....,ご覧いただきありがとうございます MAISHA マイシャ 1985年 Sadao...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他
4,2019/10/09 15:12:05,https://item.fril.jp/a06a93f70dce76664c0054b53...,LD★PARKER'S MOOD（パーカーズ・ムード）LIVE AT BRAVA,3000,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,9,0,0,1-2日後,...,2000-01-01,0,東京都,着払い,https://fril.jp/shop/22d352b7a529906589dcdbc60...,なし,https://img.fril.jp/img/124947258/l/353856770....,ご覧いただきありがとうございます PARKER S MOOD パーカーズ ムード LI...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63001,2019/07/12 20:23:30,https://item.fril.jp/4a79945d051dec14200c85503...,64 マリオテニス,500,NINTENDO64,エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト,368,2,5,4~7日で発送,...,2000-01-01,0,熊本県,送料込,https://fril.jp/shop/35f887a74a9739c8d9e5d026f...,なし,https://img.fril.jp/img/220330456/l/626462275....,任天堂64のカセットです 裏にイニシャル記載あり 箱 説明書などあります 動作確認...,"{'カテゴリ': 'エンタメ/ホビー›テレビゲーム›家庭用ゲームソフト', 'サイズ': '...",エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト
63002,2019/06/16 15:30:53,https://item.fril.jp/6bc3baba28c29a4681417ac56...,☆非売品☆switch ポケットモンスター ポケモンカードバトル クリアファイル,950,NintendoSwitch,エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト,372,1,4,3,...,2000-01-01,0,山口県,送料込,https://fril.jp/shop/5f35d399309625c1388e183a1...,なし,https://img.fril.jp/img/236315137/l/670680999....,新品未開封非売品ですソフトに付いてましたクリアファイル応募券付きです丁寧に雨対策をして台紙を...,"{'カテゴリ': 'エンタメ/ホビー›テレビゲーム›家庭用ゲームソフト', 'サイズ': '...",エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト
63003,2019/06/13 01:12:35,https://item.fril.jp/a7c498013a4fc4ac43f8cc014...,【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）,8500,NintendoSwitch,エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト,95,1,2,2,...,2000-01-01,0,東京都,送料込,https://fril.jp/shop/23707e46a8542b94dadaeff32...,なし,https://img.fril.jp/img/224384559/l/637663265....,新品で未開封です大人気商品のため在庫残りわずかです特別にクロスクリーナーも付いています受注後...,"{'カテゴリ': 'エンタメ/ホビー›テレビゲーム›家庭用ゲームソフト', 'サイズ': '...",エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト
63004,2019/06/13 02:52:50,https://item.fril.jp/a7c498013a4fc4ac43f8cc014...,【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）,8500,NintendoSwitch,エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト,95,1,2,2,...,2000-01-01,0,東京都,送料込,https://fril.jp/shop/23707e46a8542b94dadaeff32...,なし,https://img.fril.jp/img/224384559/l/637663265....,新品で未開封です大人気商品のため在庫残りわずかです特別にクロスクリーナーも付いています受注後...,"{'カテゴリ': 'エンタメ/ホビー›

In [16]:
data.category.nunique()

22

In [17]:
category2idx = {c: idx for idx, c in enumerate(data.category.unique())}
category2idx

{'エンタメ/ホビー > エンタメその他 > その他': 0,
 'エンタメ/ホビー > CD > ポップス/ロック(洋楽)': 1,
 'エンタメ/ホビー > CD > ポップス/ロック(邦楽)': 2,
 'エンタメ/ホビー > CD > アニメ': 3,
 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ': 4,
 'エンタメ/ホビー > アニメグッズ > その他': 5,
 'エンタメ/ホビー > フィギュア > アニメ/ゲーム': 6,
 'エンタメ/ホビー > フィギュア > 特撮': 7,
 'レディース > ワンピース > ひざ丈ワンピース': 8,
 'インテリア/住まい/日用品 > 文房具 > ペン/マーカー': 9,
 'スポーツ/アウトドア > フィッシング > ルアー用品': 10,
 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ぬいぐるみ': 11,
 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > 模型/プラモデル': 12,
 'メンズ > トップス > Tシャツ/カットソー(半袖/袖なし)': 13,
 'スマホ/家電/カメラ > オーディオ機器 > ヘッドフォン/イヤフォン': 14,
 'エンタメ/ホビー > トレーディングカード > シングルカード': 15,
 '楽器 > ギター > エレキギター': 16,
 'メンズ > 靴/シューズ > スニーカー': 17,
 'エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー': 18,
 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー': 19,
 'エンタメ/ホビー > テレビゲーム > 家庭用ゲーム本体': 20,
 'エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト': 21}

In [18]:
idx2category = {idx: c for idx, c in enumerate(data.category.unique())}
idx2category

{0: 'エンタメ/ホビー > エンタメその他 > その他',
 1: 'エンタメ/ホビー > CD > ポップス/ロック(洋楽)',
 2: 'エンタメ/ホビー > CD > ポップス/ロック(邦楽)',
 3: 'エンタメ/ホビー > CD > アニメ',
 4: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ',
 5: 'エンタメ/ホビー > アニメグッズ > その他',
 6: 'エンタメ/ホビー > フィギュア > アニメ/ゲーム',
 7: 'エンタメ/ホビー > フィギュア > 特撮',
 8: 'レディース > ワンピース > ひざ丈ワンピース',
 9: 'インテリア/住まい/日用品 > 文房具 > ペン/マーカー',
 10: 'スポーツ/アウトドア > フィッシング > ルアー用品',
 11: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ぬいぐるみ',
 12: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > 模型/プラモデル',
 13: 'メンズ > トップス > Tシャツ/カットソー(半袖/袖なし)',
 14: 'スマホ/家電/カメラ > オーディオ機器 > ヘッドフォン/イヤフォン',
 15: 'エンタメ/ホビー > トレーディングカード > シングルカード',
 16: '楽器 > ギター > エレキギター',
 17: 'メンズ > 靴/シューズ > スニーカー',
 18: 'エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー',
 19: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー',
 20: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲーム本体',
 21: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト'}

# テキストの前処理

In [28]:
emojis = ''
for key in emoji.UNICODE_EMOJI.keys():
    emojis += "".join(emoji.UNICODE_EMOJI[key].keys())

In [29]:
emojis[-100:]

'🇿🇹🇦🇹🇨🇹🇩🇹🇫🇹🇬🇹🇭🇹🇯🇹🇰🇹🇱🇹🇲🇹🇳🇹🇴🇹🇷🇹🇹🇹🇻🇹🇼🇹🇿🇺🇦🇺🇬🇺🇲🇺🇳🇺🇸🇺🇾🇺🇿🇻🇦🇻🇨🇻🇪🇻🇬🇻🇮🇻🇳🇻🇺🇼🇫🇼🇸🇽🇰🇾🇪🇾🇹🇿🇦🇿🇲🇿🇼🏴\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f🏴\U000e0067\U000e0062\U000e0073\U000e0063\U000e0074\U000e007f🏴\U000e0067\U000e0062\U000e0077\U000e006c\U000e0073\U000e007f'

In [30]:
puncs = string.punctuation +  "◆▼★②●☆■★【】『』「」、♪"
puncs

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~◆▼★②●☆■★【】『』「」、♪'

In [77]:
#半角から全角
def han2zen(txt):
    txt = jctconv.h2z(txt, kana=True, digit=False, ascii=False)
    txt = jctconv.z2h(txt, kana=False, digit=True, ascii=True)
    return txt

def remove_signs(txt):
    rm_signs = emojis + puncs
    for sign in rm_signs:
        txt = txt.replace(sign, " ")
        
    return txt

def get_nouns(txt):
    tokens = tkn.tokenize(txt)
    return " ".join([
        token.surface.strip()
        for token in tokens
        if token.part_of_speech.split(",")[0] == '名詞'
    ])

def clean_txt(txt):
    txt = han2zen(txt)
    txt = remove_signs(txt)
    txt_list = txt.lower().split()
    txt_list = [x for x in txt_list if len(x) > 1]
    txt = " ".join(list(Counter(txt_list)))
    return get_nouns(txt)

## 試し

In [39]:
data.title[1]

'送料無料 新品 DVD Perfume WORLD TOUR 1st 初回'

In [40]:
clean_txt(data.title[1])

'送料無料 新品 dvd perfume world tour st 初回'

In [63]:
from janome.tokenizer import Tokenizer
tkn = Tokenizer()

In [74]:
txt = '送料無料 新品 DVD Perfume WORLD TOUR 1st 初回'
tokens = tkn.tokenize(txt)

In [75]:
for token in tokens:
    print(token.surface.strip())
    print(token.part_of_speech.split(","))

送料
['名詞', '一般', '*', '*']
無料
['名詞', '一般', '*', '*']

['記号', '空白', '*', '*']
新品
['名詞', '一般', '*', '*']

['記号', '空白', '*', '*']
DVD
['名詞', '固有名詞', '組織', '*']

['記号', '空白', '*', '*']
Perfume
['名詞', '固有名詞', '組織', '*']

['記号', '空白', '*', '*']
WORLD
['名詞', '固有名詞', '組織', '*']

['記号', '空白', '*', '*']
TOUR
['名詞', '固有名詞', '組織', '*']

['記号', '空白', '*', '*']
1
['名詞', '数', '*', '*']
st
['名詞', '固有名詞', '組織', '*']

['記号', '空白', '*', '*']
初回
['名詞', '一般', '*', '*']


### 試し終わり

In [81]:
data.title.head()

0       テレカ 未使用品 仮面ライダー 生誕20周年 東映ビデオ販売 販促用非売品
1       送料無料 新品 DVD Perfume WORLD TOUR 1st 初回
2                      ☆ウルトラマン☆レーザーディスク ジャンク？
3         MAISHA （マイシャ）Sadao Watanabe　渡辺貞夫★LD
4    LD★PARKER'S MOOD（パーカーズ・ムード）LIVE AT BRAVA
Name: title, dtype: object

In [79]:
data["cln_title"] = data.title.apply(clean_txt)
data["cln_title"].head()

0    テレカ 使用 品 仮面ライダー 生誕 周 年 東映 ビデオ 販売 販促 用 非売品
1        送料 無料 新品 dvd perfume world tour st 初回
2                        ウルトラマン レーザー ディスク ジャンク
3          maisha マイシャ sadao watanabe 渡辺 貞夫 ld
4       ld parker mood パーカーズ・ムード live at brava
Name: cln_title, dtype: object

In [82]:
data.description.head()

0      テレカ 未使用 50度数 レア 仮面ライダー 誕生20周年 レーザーディスク販売時の販促...
1       新品 DVD Perfume WORLD TOUR 1st 初回プレス盤 ご覧頂きあり...
2      ウルトラマンのレーザーディスクです  再生機器が無いのと 未開封のようですので現状渡しで...
3      ご覧いただきありがとうございます  MAISHA  マイシャ 1985年   Sadao...
4      ご覧いただきありがとうございます  PARKER S MOOD パーカーズ ムード LI...
Name: description, dtype: object

In [80]:
data["cln_description"] = data.description.apply(clean_txt)
data["cln_description"].head()

0    テレカ 使用 度数 レア 仮面ライダー 誕生 周 年 レーザー ディスク 販売 時 販促 用...
1    新品 dvd perfume world tour st 初回 プレス 盤 ご覧 送料 無料...
2    ウルトラマン レーザー ディスク 再生 機器 の 開封 よう 現状 渡し ジャンク 品 お願...
3    ご覧 いただき maisha マイシャ sadao watanabe 渡辺 貞夫 ld レー...
4    ご覧 いただき parker mood パーカーズ ムード live at bravas c...
Name: cln_description, dtype: object

In [89]:
last_data = data[data["cln_title"].str.split().str.len() > 2].reset_index(drop=True)
last_data.head()

,date,item_url,title,price,brand,category,good,bad,normal,ship_days,...,prefecture,delivery_fee,seller_url,size,picture_url,description,details_dict,index,cln_title,cln_description
0,2019/10/09 15:12:05,https://item.fril.jp/56e1889edb4373b060739c876...,テレカ 未使用品 仮面ライダー 生誕20周年 東映ビデオ販売 販促用非売品,2500,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,37,0,0,1-2日後,...,神奈川県,送料込,https://fril.jp/shop/d16cd0f0dd452bb98787cc349...,なし,https://img.fril.jp/img/152839505/l/513856974....,テレカ 未使用 50度数 レア 仮面ライダー 誕生20周年 レーザーディスク販売時の販促...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他,テレカ 使用 品 仮面ライダー 生誕 周 年 東映 ビデオ 販売 販促 用 非売品,テレカ 使用 度数 レア 仮面ライダー 誕生 周 年 レーザー ディスク 販売 時 販促 用...
1,2019/10/09 15:12:05,https://item.fril.jp/ac2ac2e9968e30e868e1cc87b...,送料無料 新品 DVD Perfume WORLD TOUR 1st 初回,4800,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,60,0,0,1-2日後,...,京都府,送料込,https://fril.jp/shop/a1c14809b40f535a4ec33b213...,なし,https://img.fril.jp/img/154299175/l/439614345....,新品 DVD Perfume WORLD TOUR 1st 初回プレス盤 ご覧頂きあり...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他,送料 無料 新品 dvd perfume world tour st 初回,新品 dvd perfume world tour st 初回 プレス 盤 ご覧 送料 無料...
2,2019/10/09 15:12:05,https://item.fril.jp/797582a80b28b6c07bfe84c03...,☆ウルトラマン☆レーザーディスク ジャンク？,1980,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,87,0,1,1-2日後,...,岡山県,送料込,https://fril.jp/shop/5d3613f833a4ee3a15e9dd72f...,なし,https://img.fril.jp/img/244479040/l/693281987....,ウルトラマンのレーザーディスクです 再生機器が無いのと 未開封のようですので現状渡しで...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他,ウルトラマン レーザー ディスク ジャンク,ウルトラマン レーザー ディスク 再生 機器 の 開封 よう 現状 渡し ジャンク 品 お願...
3,2019/10/09 15:12:05,https://item.fril.jp/5da5209c6bd09ceef11d1ed2d...,MAISHA （マイシャ）Sadao Watanabe 渡辺貞夫★LD,2600,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,9,0,0,1-2日後,...,東京都,着払い,https://fril.jp/shop/22d352b7a529906589dcdbc60...,なし,https://img.fril.jp/img/124947216/l/353856650....,ご覧いただきありがとうございます MAISHA マイシャ 1985年 Sadao...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他,maisha マイシャ sadao watanabe 渡辺 貞夫 ld,ご覧 いただき maisha マイシャ sadao watanabe 渡辺 貞夫 ld レー...
4,2019/10/09 15:12:05,https://item.fril.jp/a06a93f70dce76664c0054b53...,LD★PARKER'S MOOD（パーカーズ・ムード）LIVE AT BRAVA,3000,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,9,0,0,1-2日後,...,東京都,着払い,https://fril.jp/shop/22d352b7a529906589dcdbc60...,なし,https://img.fril.jp/img/124947258/l/353856770....,ご覧いただきありがとうございます PARKER S MOOD パーカーズ ムード LI...,"{'カテゴリ': 'エンタメ/ホビー›エンタメその他›その他', 'サイズ': 'なし', ...",エンタメ/ホビー > エンタメその他 > その他,ld parker mood パーカーズ・ムード live at brava,ご覧 いただき parker mood パーカーズ ムード live at bravas c...


# Word2vec(テキストを数値に置き換える)

In [104]:
sentences = list((last_data["cln_title"] + " " + last_data["cln_description"]).str.split())

In [105]:
sentences[0]

['テレカ',
 '使用',
 '品',
 '仮面ライダー',
 '生誕',
 '周',
 '年',
 '東映',
 'ビデオ',
 '販売',
 '販促',
 '用',
 '非売品',
 'テレカ',
 '使用',
 '度数',
 'レア',
 '仮面ライダー',
 '誕生',
 '周',
 '年',
 'レーザー',
 'ディスク',
 '販売',
 '時',
 '販促',
 '用',
 '非売品',
 '質問',
 '等',
 'コメント',
 '連絡',
 'お願い']

In [106]:
from gensim.models import word2vec
import os

In [107]:
dim = 100
min_count = 0
sg = 1 #スキップグラム

In [108]:
len(sentences)

57587

In [115]:
wv_model = word2vec.Word2Vec(sentences, vector_size=dim, min_count=min_count, workers=os.cpu_count(), sg=sg)

In [116]:
wv_model

In [117]:
txt = sentences[0]
txt

['テレカ',
 '使用',
 '品',
 '仮面ライダー',
 '生誕',
 '周',
 '年',
 '東映',
 'ビデオ',
 '販売',
 '販促',
 '用',
 '非売品',
 'テレカ',
 '使用',
 '度数',
 'レア',
 '仮面ライダー',
 '誕生',
 '周',
 '年',
 'レーザー',
 'ディスク',
 '販売',
 '時',
 '販促',
 '用',
 '非売品',
 '質問',
 '等',
 'コメント',
 '連絡',
 'お願い']

In [119]:
wv_model.wv[txt].shape

(33, 100)

In [120]:
wv_model.wv[txt]

array([[-0.07941913,  0.28845826,  0.419991  , ..., -0.18437368,
        -0.02969903,  0.14261287],
       [-0.29597875,  0.41098756, -0.1971992 , ..., -0.3987101 ,
         0.02513241, -0.29389203],
       [-0.25508934,  0.23448525, -0.24766319, ...,  0.31786838,
         0.05447758, -0.37740785],
       ...,
       [-0.10609787,  0.38356817,  0.1089419 , ..., -0.64569193,
         0.08629664, -0.5496638 ],
       [ 0.02084304,  0.4900716 , -0.24273105, ..., -0.6114714 ,
         0.5214834 ,  0.00663191],
       [-0.76160175,  0.44910547, -0.25001132, ..., -0.11483093,
         0.20588218, -0.28018138]], dtype=float32)

In [122]:
vectors = pd.DataFrame(wv_model.wv[txt])
vectors.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.079419,0.288458,0.419991,0.425605,-0.279482,-0.172935,0.310554,0.146567,-0.137458,-0.206263,...,0.163352,-0.412728,0.152418,-0.291081,-0.132889,-0.002497,0.115111,-0.184374,-0.029699,0.142613
1,-0.295979,0.410988,-0.197199,0.371405,-0.688679,-0.107633,-0.055957,0.408272,0.132077,-0.223432,...,0.541762,0.274440,-0.307476,0.064673,-0.342179,-0.061519,0.011549,-0.398710,0.025132,-0.293892
2,-0.255089,0.234485,-0.247663,0.280301,-0.564123,-0.041900,0.088610,0.507451,0.309430,-0.683358,...,0.340536,-0.128241,0.053477,-0.012572,-0.218425,-0.230008,0.110047,0.317868,0.054478,-0.377408
3,0.711238,0.624642,-0.437129,0.460781,-1.010033,0.632259,0.829911,0.407313,-0.451621,-1.003438,...,-0.176479,0.102038,-0.623674,-0.465797,-0.038493,0.089080,-0.091714,-0.977706,0.360313,0.245847
4,-0.021840,0.424107,-0.070359,0.106362,-0.344100,-0.404261,0.174742,-0.060374,-0.369431,-0.286251,...,0.286327,-0.472994,0.094325,-0.313849,0.013040,0.223038,-0.273271,-0.200438,-0.381480,0.451618


In [123]:
sentence_df = pd.DataFrame(txt, columns=["title"])
vec_df = pd.concat([
    sentence_df,
    vectors
], axis=1)
vec_df

,title,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,テレカ,-0.079419,0.288458,0.419991,0.425605,-0.279482,-0.172935,0.310554,0.146567,-0.137458,...,0.163352,-0.412728,0.152418,-0.291081,-0.132889,-0.002497,0.115111,-0.184374,-0.029699,0.142613
1,使用,-0.295979,0.410988,-0.197199,0.371405,-0.688679,-0.107633,-0.055957,0.408272,0.132077,...,0.541762,0.274440,-0.307476,0.064673,-0.342179,-0.061519,0.011549,-0.398710,0.025132,-0.293892
2,品,-0.255089,0.234485,-0.247663,0.280301,-0.564123,-0.041900,0.088610,0.507451,0.309430,...,0.340536,-0.128241,0.053477,-0.012572,-0.218425,-0.230008,0.110047,0.317868,0.054478,-0.377408
3,仮面ライダー,0.711238,0.624642,-0.437129,0.460781,-1.010033,0.632259,0.829911,0.407313,-0.451621,...,-0.176479,0.102038,-0.623674,-0.465797,-0.038493,0.089080,-0.091714,-0.977706,0.360313,0.245847
4,生誕,-0.021840,0.424107,-0.070359,0.106362,-0.344100,-0.404261,0.174742,-0.060374,-0.369431,...,0.286327,-0.472994,0.094325,-0.313849,0.013040,0.223038,-0.273271,-0.200438,-0.381480,0.451618
5,周,0.262582,0.188656,0.834865,-0.138590,-0.236554,-0.620509,0.289947,-0.230979,-0.733069,...,-0.479419,-0.761774,-0.318846,-0.162273,0.132614,0.375688,-0.544749,-0.193199,-0.272012,1.018552
6,年,0.188015,0.620170,-0.004013,0.163943,-0.166323,-0.651990,0.020851,-0.317293,-0.864252,...,-0.099777,-0.138481,-0.014327,0.373802,0.032596,0.508967,-0.437000,-0.371632,-0.296975,0.453146
7,東映,-0.079860,0.553905,0.753948,0.334127,0.023983,0.066550,0.834507,0.482427,-0.172033,...,0.143331,-0.736108,0.577474,-0.162804,-0.115559,0.574606,0.586080,-0.244093,-0.056581,0.180660
8,ビデオ,0.120274,0.426112,0.473858,0.473891,-0.008601,-0.035444,0.436860,0.226582,-0.109202,...,0.274394,-0.428621,0.480280,-0.361101,-0.523141,0.241499,-0.018492,0.096123,0.093271,0.266162
9,販売,-0.059769,0.080740,0.352264,0.000838,-0.621162,-0.412284,0.514122,0.454299,-0.194178,...,0.401982,-0.293048,-0.352953,0.149787,-0.071143,0.165018,-0.565540,0.068866,0.089275,-0.208339


In [126]:
" ".join(vec_df.title)

'テレカ 使用 品 仮面ライダー 生誕 周 年 東映 ビデオ 販売 販促 用 非売品 テレカ 使用 度数 レア 仮面ライダー 誕生 周 年 レーザー ディスク 販売 時 販促 用 非売品 質問 等 コメント 連絡 お願い'

In [127]:
vec_df.select_dtypes(exclude="object").mean()

0    -0.001803
1     0.386109
2     0.280038
3     0.360791
4    -0.430167
        ...   
95    0.061579
96   -0.181253
97   -0.260805
98   -0.014427
99    0.066207
Length: 100, dtype: float32

In [128]:
vec_data = [" ".join(vec_df.title)] + list(vec_df.select_dtypes(exclude="object").mean())
pd.DataFrame([vec_data])

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,テレカ 使用 品 仮面ライダー 生誕 周 年 東映 ビデオ 販売 販促 用 非売品 テレカ ...,-0.001803,0.386109,0.280038,0.360791,-0.430167,-0.199916,0.340166,0.176716,-0.240184,...,0.074838,-0.270056,-0.041885,-0.201225,-0.087545,0.061579,-0.181253,-0.260805,-0.014427,0.066207


In [129]:
last_data.category

0             エンタメ/ホビー > エンタメその他 > その他
1             エンタメ/ホビー > エンタメその他 > その他
2             エンタメ/ホビー > エンタメその他 > その他
3             エンタメ/ホビー > エンタメその他 > その他
4             エンタメ/ホビー > エンタメその他 > その他
                     ...              
57582    エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト
57583    エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト
57584    エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト
57585    エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト
57586    エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト
Name: category, Length: 57587, dtype: object

In [130]:
pd.DataFrame([wv_model.wv[txt].mean(axis=0)])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.001803,0.386109,0.280038,0.360791,-0.430167,-0.199916,0.340166,0.176716,-0.240184,-0.265169,...,0.074838,-0.270056,-0.041885,-0.201225,-0.087545,0.061579,-0.181253,-0.260805,-0.014427,0.066207


In [131]:
vectors = [wv_model.wv[txt].mean(axis=0) for txt in sentences]



In [138]:
pd.DataFrame(vectors)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.001803,0.386109,0.280038,0.360791,-0.430167,-0.199916,0.340166,0.176716,-0.240184,-0.265169,...,0.074838,-0.270056,-0.041885,-0.201225,-0.087545,0.061579,-0.181253,-0.260805,-0.014427,0.066207
1,-0.169152,0.286232,0.020361,0.344314,-0.294411,-0.139106,0.185835,0.258191,-0.161622,-0.234284,...,0.165325,-0.113530,0.092899,-0.110177,-0.160012,0.059912,0.013495,-0.136159,0.109714,-0.110257
2,0.098074,0.239152,-0.062853,0.405201,-0.464684,-0.311791,0.610618,0.247927,-0.115615,-0.360531,...,0.024081,-0.081465,-0.088223,-0.315615,-0.111640,0.050890,-0.036679,-0.119055,0.162180,-0.184905
3,-0.090704,0.408745,-0.051902,0.411357,-0.307405,-0.078232,0.378460,0.192763,-0.133990,-0.132595,...,0.043369,-0.197842,0.122191,-0.132870,-0.220918,0.161490,-0.066175,-0.054948,0.004224,-0.109190
4,-0.164913,0.363681,0.004960,0.421470,-0.328906,-0.089717,0.265595,0.153499,-0.165161,-0.152953,...,0.103922,-0.210058,0.078645,-0.160797,-0.268905,0.178695,-0.031769,-0.081044,0.052264,-0.113223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57582,-0.240966,0.412977,0.129199,0.450259,-0.249033,-0.252707,0.418949,0.353813,-0.335997,-0.233374,...,0.138354,-0.176785,0.028605,-0.101459,-0.179845,0.024601,-0.209750,-0.158623,0.041532,-0.016270
57583,-0.196438,0.573653,0.273625,0.487179,-0.422998,-0.157598,0.322935,0.222371,-0.151824,-0.076868,...,0.027537,-0.100317,0.034805,-0.155047,-0.203091,-0.051203,-0.142300,-0.189416,0.056679,-0.011157
57584,-0.166506,0.420662,0.183141,0.323204,-0.391473,-0.225267,0.215480,0.255799,-0.201760,-0.294162,...,0.301918,-0.133339,-0.132375,-0.210602,-0.181329,0.041621,-0.182638,-0.093493,0.052576,-0.008815
57585,-0.166506,0.420662,0.183141,0.323204,-0.391473,-0.225267,0.215480,0.255799,-0.201760,-0.294162,...,0.301918,-0.133339,-0.132375,-0.210602,-0.181329,0.041621,-0.182638,-0.093493,0.052576,-0.008815


In [140]:
last_data.category.apply(lambda x: category2idx[x])

0         0
1         0
2         0
3         0
4         0
         ..
57582    21
57583    21
57584    21
57585    21
57586    21
Name: category, Length: 57587, dtype: int64

In [142]:
X = pd.DataFrame(vectors)
y = last_data.category.apply(lambda x: category2idx[x]).values

In [143]:
len(y) == len(vectors)

True

In [144]:
X

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.001803,0.386109,0.280038,0.360791,-0.430167,-0.199916,0.340166,0.176716,-0.240184,-0.265169,...,0.074838,-0.270056,-0.041885,-0.201225,-0.087545,0.061579,-0.181253,-0.260805,-0.014427,0.066207
1,-0.169152,0.286232,0.020361,0.344314,-0.294411,-0.139106,0.185835,0.258191,-0.161622,-0.234284,...,0.165325,-0.113530,0.092899,-0.110177,-0.160012,0.059912,0.013495,-0.136159,0.109714,-0.110257
2,0.098074,0.239152,-0.062853,0.405201,-0.464684,-0.311791,0.610618,0.247927,-0.115615,-0.360531,...,0.024081,-0.081465,-0.088223,-0.315615,-0.111640,0.050890,-0.036679,-0.119055,0.162180,-0.184905
3,-0.090704,0.408745,-0.051902,0.411357,-0.307405,-0.078232,0.378460,0.192763,-0.133990,-0.132595,...,0.043369,-0.197842,0.122191,-0.132870,-0.220918,0.161490,-0.066175,-0.054948,0.004224,-0.109190
4,-0.164913,0.363681,0.004960,0.421470,-0.328906,-0.089717,0.265595,0.153499,-0.165161,-0.152953,...,0.103922,-0.210058,0.078645,-0.160797,-0.268905,0.178695,-0.031769,-0.081044,0.052264,-0.113223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57582,-0.240966,0.412977,0.129199,0.450259,-0.249033,-0.252707,0.418949,0.353813,-0.335997,-0.233374,...,0.138354,-0.176785,0.028605,-0.101459,-0.179845,0.024601,-0.209750,-0.158623,0.041532,-0.016270
57583,-0.196438,0.573653,0.273625,0.487179,-0.422998,-0.157598,0.322935,0.222371,-0.151824,-0.076868,...,0.027537,-0.100317,0.034805,-0.155047,-0.203091,-0.051203,-0.142300,-0.189416,0.056679,-0.011157
57584,-0.166506,0.420662,0.183141,0.323204,-0.391473,-0.225267,0.215480,0.255799,-0.201760,-0.294162,...,0.301918,-0.133339,-0.132375,-0.210602,-0.181329,0.041621,-0.182638,-0.093493,0.052576,-0.008815
57585,-0.166506,0.420662,0.183141,0.323204,-0.391473,-0.225267,0.215480,0.255799,-0.201760,-0.294162,...,0.301918,-0.133339,-0.132375,-0.210602,-0.181329,0.041621,-0.182638,-0.093493,0.052576,-0.008815


In [145]:
y

array([ 0,  0,  0, ..., 21, 21, 21])

# データの学習

In [146]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import lightgbm as lgbm
import numpy as np

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [149]:
len(X_train)
len(X_test)
len(y_train)
len(y_test)

19004

In [150]:
len(idx2category)

22

In [151]:
params = {
    "objective": "multiclass",
    "metric": "multi_logloss",
    "verbosity": -1,
    "num_class": len(idx2category),
    "boosting_type": "gbdt"
}
params

{'objective': 'multiclass',
 'metric': 'multi_logloss',
 'verbosity': -1,
 'num_class': 22,
 'boosting_type': 'gbdt'}

In [152]:
lgb_train = lgbm.Dataset(X_train, y_train)
lgb = lgbm.train(params, lgb_train)

# 精度を評価する

In [164]:
np.argmax(lgb.predict(X_test)[0])

10

In [165]:
idx2category[10]

'スポーツ/アウトドア > フィッシング > ルアー用品'

In [166]:
y_train[0]

17

↑ 間違ってる
ほんとの答えは

In [167]:
idx2category[y_train[0]]

'メンズ > 靴/シューズ > スニーカー'

In [155]:
prediction = np.array([np.argmax(pr) for pr in lgb.predict(X_test)])

In [156]:
len(prediction) == len(X_test)

True

In [158]:
prediction

array([10,  4,  6, ..., 17, 10,  9])

In [157]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.86      0.69      0.77       372
           1       0.96      0.96      0.96      1124
           2       0.87      0.86      0.86       382
           3       0.96      0.96      0.96       689
           4       0.80      0.90      0.84      3278
           5       0.70      0.38      0.50       395
           6       0.83      0.92      0.87      2000
           7       0.87      0.89      0.88       900
           8       0.99      0.98      0.98       393
           9       0.99      0.89      0.94       313
          10       0.99      0.96      0.97       685
          11       0.80      0.54      0.65       443
          12       0.89      0.80      0.84       456
          13       0.93      0.90      0.92       389
          14       0.98      0.97      0.97       487
          15       0.99      0.93      0.96       660
          16       0.99      0.95      0.97       372
          17       0.99    

In [123]:
idx2category

{0: 'エンタメ/ホビー > エンタメその他 > その他',
 1: 'エンタメ/ホビー > CD > ポップス/ロック(洋楽)',
 2: 'エンタメ/ホビー > CD > ポップス/ロック(邦楽)',
 3: 'エンタメ/ホビー > CD > アニメ',
 4: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ',
 5: 'エンタメ/ホビー > アニメグッズ > その他',
 6: 'エンタメ/ホビー > フィギュア > アニメ/ゲーム',
 7: 'エンタメ/ホビー > フィギュア > 特撮',
 8: 'レディース > ワンピース > ひざ丈ワンピース',
 9: 'インテリア/住まい/日用品 > 文房具 > ペン/マーカー',
 10: 'スポーツ/アウトドア > フィッシング > ルアー用品',
 11: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ぬいぐるみ',
 12: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > 模型/プラモデル',
 13: 'メンズ > トップス > Tシャツ/カットソー(半袖/袖なし)',
 14: 'スマホ/家電/カメラ > オーディオ機器 > ヘッドフォン/イヤフォン',
 15: 'エンタメ/ホビー > トレーディングカード > シングルカード',
 16: '楽器 > ギター > エレキギター',
 17: 'メンズ > 靴/シューズ > スニーカー',
 18: 'エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー',
 19: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー',
 20: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲーム本体',
 21: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト'}

# 学習済みモデルの保存

In [124]:
# target = ["NIKE AIR FORCE 1 LOW QS PUERTO RICO 2020"]
target = ["ニューバランス Cross Country 25.5cm M320"]
target = ["✨新品✨オニツカタイガー ❇️25.0cm"]
target = ["ビームス　ナイキ　REACT PRESTO DHARMA"]
prediction = pipeline.predict(target)

prediction

array([17])

In [125]:
idx2category[prediction[0]]

'メンズ > 靴/シューズ > スニーカー'

In [126]:
import pickle

In [127]:
with open("rdmf.pickle", mode="wb") as f:
    pickle.dump(pipeline, f)

In [129]:
#読み込むときは
with open("rdmf.pickle", mode="rb") as ff:
    model = pickle.load(ff)

In [130]:
model.predict(target)

array([17])

## カテゴリーも保存しておく

In [131]:
idx2category

{0: 'エンタメ/ホビー > エンタメその他 > その他',
 1: 'エンタメ/ホビー > CD > ポップス/ロック(洋楽)',
 2: 'エンタメ/ホビー > CD > ポップス/ロック(邦楽)',
 3: 'エンタメ/ホビー > CD > アニメ',
 4: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ',
 5: 'エンタメ/ホビー > アニメグッズ > その他',
 6: 'エンタメ/ホビー > フィギュア > アニメ/ゲーム',
 7: 'エンタメ/ホビー > フィギュア > 特撮',
 8: 'レディース > ワンピース > ひざ丈ワンピース',
 9: 'インテリア/住まい/日用品 > 文房具 > ペン/マーカー',
 10: 'スポーツ/アウトドア > フィッシング > ルアー用品',
 11: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ぬいぐるみ',
 12: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > 模型/プラモデル',
 13: 'メンズ > トップス > Tシャツ/カットソー(半袖/袖なし)',
 14: 'スマホ/家電/カメラ > オーディオ機器 > ヘッドフォン/イヤフォン',
 15: 'エンタメ/ホビー > トレーディングカード > シングルカード',
 16: '楽器 > ギター > エレキギター',
 17: 'メンズ > 靴/シューズ > スニーカー',
 18: 'エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー',
 19: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー',
 20: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲーム本体',
 21: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト'}

In [132]:
pd.DataFrame([
    {"k": k, "v": v}
    for k, v in idx2category.items()
]).to_csv("idx2category.csv", index=False)